Dataset Link : https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html

Pretrained Model : https://huggingface.co/transformers/v3.3.1/model_doc/distilbert.html

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 1.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 981 kB 43.6 MB/s 
     |████████████████████████████████| 263 kB 45.0 MB/s 
     |████████████████████████████████| 2.8 MB 35.9 MB/s 
     |████████████████████████████████| 1.2 MB 44.2 MB/s 
     |████████████████████████████████| 468 kB 44.9 MB/s 
     |████████████████████████████████| 596 kB 40.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 
  Created wheel for ktrain: filename=ktrain-0.29.3-py3-none-any.whl size=25295410 sha256=cdb770e11cdeb6c00729ad600e7f5a0c375dc4b2eb920d2d6c19f8af20ff3a44
  Stored in directory: /root/.cache/pip/wheels/04/d5/2a/1a1826e16685841c68a43c31c8a892452b105f9467f64e49e6
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9931 sha256=0f7c8bf8329df766

In [ ]:
%reload_ext autoreload
%autoreload
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0";

In [ ]:
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups

In [ ]:
categories = ['alt.atheism','soc.religion.christian','comp.graphics','sci.med','rec.sport.baseball']


In [ ]:
train = fetch_20newsgroups(
    subset = 'train',
    categories = categories,
    shuffle = True,
    random_state = 40
)    

In [ ]:
test = fetch_20newsgroups(
    subset = 'test',
    categories = categories,
    shuffle = True,
    random_state = 40
)   

In [ ]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
test.data

['From: banschbach@vms.ocom.okstate.edu\nSubject: Re: Candida(yeast) Bloom, Fact or Fiction\nLines: 68\nNntp-Posting-Host: vms.ocom.okstate.edu\nOrganization: OSU College of Osteopathic Medicine\n\nIn article <1rjifg$bgm@hsdndev.harvard.edu>, rind@enterprise.bih.harvard.edu (David Rind) writes:\n> In article <1993Apr26.174538.1@vms.ocom.okstate.edu>\n>  banschbach@vms.ocom.okstate.edu writes:\n>>oxygen(just like it does in the vagina).  As much stuff as there is in the \n>>lay press about L. acidophilus and vaginal yeast infections, I\'m really \n>>amazed that someone has not done a clinical trial yet to check it out.\n> \n> I\'ve mentioned this study a couple of times now: Ingestion of yogurt\n> containing Lactobacillus acidophilus as prophylaxis for candidal\n> vaginitis, Annals of Internal Medicine, 3/1/92 116(5):353-7.  Do you\n> have a problem with the study because they used yogurt rather than\n> capsules of lactobacillus (even though it had positive results)?\n> \n> The study wa

In [ ]:
test.target_names

['alt.atheism',
 'comp.graphics',
 'rec.sport.baseball',
 'sci.med',
 'soc.religion.christian']

In [ ]:
X_train = train.data
y_train = train.target

X_test = test.data
y_test = test.target

In [ ]:
len(X_train),len(X_test)

(2854, 1899)

In [ ]:
#Building ML Model with Transformer
Model_name = 'distilbert-base-uncased'

In [ ]:
trans = text.Transformer(Model_name,maxlen = 512,class_names = categories)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
train_data = trans.preprocess_train(X_train,y_train)
test_data = trans.preprocess_test(X_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 291
	95percentile : 820
	99percentile : 1757


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 323
	95percentile : 894
	99percentile : 2394


In [ ]:
model = trans.get_classifier()

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:431: UserWarning: The method preprocess_train was never called. You can disable this warning by setting preprocess_train_called=True.
  warnings.warn('The method preprocess_train was never called. You can disable this warning by setting preprocess_train_called=True.')


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
learner = ktrain.get_learner(model,train_data=train_data,val_data = test_data,batch_size = 16)

In [ ]:
learner.fit_onecycle(1e-4,1)



begin training using onecycle policy with max lr of 0.0001...
179/179 [==============================] - 419s 2s/step - loss: 0.5444 - accuracy: 0.8167 - val_loss: 0.2434 - val_accuracy: 0.9179


In [ ]:
learner.validate(class_names = categories)

                        precision    recall  f1-score   support

           alt.atheism       0.88      0.73      0.80       319
soc.religion.christian       0.95      0.94      0.95       389
         comp.graphics       0.95      0.99      0.97       397
               sci.med       0.97      0.93      0.95       396
    rec.sport.baseball       0.84      0.96      0.90       398

              accuracy                           0.92      1899
             macro avg       0.92      0.91      0.91      1899
          weighted avg       0.92      0.92      0.92      1899



array([[233,   1,  10,   8,  67],
       [ 12, 367,   4,   4,   2],
       [  0,   4, 393,   0,   0],
       [ 10,  11,   5, 368,   2],
       [ 11,   4,   1,   0, 382]])

In [ ]:
learner.view_top_losses(n = 5,preproc=trans)

----------
id:276 | loss:5.59 | true:rec.sport.baseball | pred:comp.graphics)

----------
id:865 | loss:5.29 | true:comp.graphics | pred:soc.religion.christian)

----------
id:978 | loss:5.2 | true:sci.med | pred:alt.atheism)

----------
id:1616 | loss:5.19 | true:soc.religion.christian | pred:rec.sport.baseball)

----------
id:229 | loss:5.16 | true:alt.atheism | pred:comp.graphics)



This is our worst classified records

In [ ]:
#Predict on new data
predictor = ktrain.get_predictor(learner.model,preproc=trans)

In [ ]:
x = "My Friend is suffering from Cancer"

In [ ]:
predictor.predict(x)

'sci.med'

Hurray we got the correct prediction

In [ ]:
#Saving the model
predictor.save('distilber-model')